In [121]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

In [122]:

    data = pd.read_csv("data.csv", encoding='latin1')
data.head(4)

,SoilMoisture,Rainfall,Humidity,Temperature,GrowthStage,NitrogenNeeded,PhosphorusNeeded,PotassiumNeeded,SoilpH
0,450.0,20.0,65.0,25.0,1.0,50.0,30.0,40.0,6.5
1,460.0,25.0,70.0,24.0,1.0,55.0,32.0,42.0,6.8
2,470.0,30.0,75.0,23.0,2.0,60.0,35.0,45.0,7.0
3,480.0,35.0,80.0,22.0,2.0,65.0,37.0,47.0,6.4


In [126]:
X = data[['SoilMoisture', 'Rainfall', 'Humidity', 'Temperature', 'GrowthStage']]
y = data[['NitrogenNeeded', 'PhosphorusNeeded', 'PotassiumNeeded']]


In [127]:
print("Missing values in features:")
print(X.isnull().sum())
print("Missing values in targets:")
print(y.isnull().sum())

Missing values in features:
SoilMoisture    20
Rainfall        20
Humidity        20
Temperature     20
GrowthStage     20
dtype: int64
Missing values in targets:
NitrogenNeeded      20
PhosphorusNeeded    20
PotassiumNeeded     20
dtype: int64


In [128]:
imputer_X = SimpleImputer(strategy='mean')
X_imputed = imputer_X.fit_transform(X)

imputer_y = SimpleImputer(strategy='mean')
y_imputed = imputer_y.fit_transform(y)

In [129]:
optimal_nutrients = {
    1: {'Nitrogen': 10, 'Phosphorus': 5, 'Potassium': 8},  # Example values for growth stage 1
    2: {'Nitrogen': 15, 'Phosphorus': 7, 'Potassium': 12}, # Example values for growth stage 2
    3: {'Nitrogen': 20, 'Phosphorus': 10, 'Potassium': 15}, # Example values for growth stage 3
    # Add more stages as needed
}

In [130]:

X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_imputed, test_size=0.2, random_state=42)


In [131]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [132]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [133]:
model.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=42)

In [134]:
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error on Test Data: {mae:.2f}")

Mean Absolute Error on Test Data: 0.76


In [135]:
test_data = pd.DataFrame([[40, 10, 65, 26, 2]], columns=['SoilMoisture', 'Rainfall', 'Humidity', 'Temperature', 'GrowthStage'])


In [136]:
test_data_imputed = imputer_X.transform(test_data)
test_data_scaled = scaler.transform(test_data_imputed)


In [137]:
prediction = model.predict(test_data_scaled)
predicted_nitrogen = prediction[0][0]
predicted_phosphorus = prediction[0][1]
predicted_potassium = prediction[0][2]


In [144]:
test_data_np = test_data.to_numpy()
test_data_scaled = scaler.transform(test_data_np)


In [145]:
current_nutrients = {'Nitrogen': 8, 'Phosphorus': 4, 'Potassium': 7}

# Determine the growth stage
growth_stage = int(test_data.iloc[0]['GrowthStage'])

In [146]:
optimal = optimal_nutrients.get(growth_stage, {'Nitrogen': 0, 'Phosphorus': 0, 'Potassium': 0})
optimal_nitrogen = optimal['Nitrogen']
optimal_phosphorus = optimal['Phosphorus']
optimal_potassium = optimal['Potassium']

In [147]:
def generate_alert(predicted, current, optimal):
    if current < optimal:
        return "Increase" if predicted < optimal else "Optimal"
    elif current > optimal:
        return "Decrease" if predicted > optimal else "Optimal"
    else:
        return "Optimal"

alert_nitrogen = generate_alert(predicted_nitrogen, current_nutrients['Nitrogen'], optimal_nitrogen)
alert_phosphorus = generate_alert(predicted_phosphorus, current_nutrients['Phosphorus'], optimal_phosphorus)
alert_potassium = generate_alert(predicted_potassium, current_nutrients['Potassium'], optimal_potassium)

In [148]:
print(f"Predicted Nitrogen Need: {predicted_nitrogen:.2f} grams/m² (Alert: {alert_nitrogen})")
print(f"Predicted Phosphorus Need: {predicted_phosphorus:.2f} grams/m² (Alert: {alert_phosphorus})")
print(f"Predicted Potassium Need: {predicted_potassium:.2f} grams/m² (Alert: {alert_potassium})")

Predicted Nitrogen Need: 57.90 grams/m² (Alert: Optimal)
Predicted Phosphorus Need: 33.60 grams/m² (Alert: Optimal)
Predicted Potassium Need: 43.60 grams/m² (Alert: Optimal)
